# A Simple Model
This notebook demonstrates Robust Decision Making (RDM) with [Rhodium](https://github.com/Project-Platypus/Rhodium/) using a simple Excel model. 

## Setting Things Up
First, lets import all the packages we'll be using.

In [1]:
from rhodium import *

# data analysis tools
import numpy as np
import pandas as pd

# additonal excel tools
import xlwings as xw

# plotting tools
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# for evaluating efficienty
import time

From the `MODEL` worksheet, we read in the model data. Note that the second column contains Excel formulas, but Pandas is only able to show the values after they're evaluated. If we want to read the actual formulas, we'll have to manually use Openpyxl and get the cell contents.

## Case #1: Driving Excel Models with `xlwings`

In [2]:
wb = xw.Book('SimpleModel.xlsx')         # connect to existing workbook
model_sheet    = wb.sheets['MODEL']      # defines the model
input_sheet    = wb.sheets['INPUTS']     # contains input data
output_sheet   = wb.sheets['OUTPUTS']    # aggregates model outputs
database_sheet = wb.sheets['DATABASE']   # stores optimization results

In [3]:
# read input data
input_labels = input_sheet.range('A2').expand('down').value
input_values = input_sheet.range('B2:B11').value 
input_mins   = input_sheet.range('C2:C11').value
input_maxs   = input_sheet.range('D2:D11').value

# Example: setting values
# inputs_sheet.range('B2:B11').options(transpose=True).value = [0, 0., 0.]

In [4]:
# read (initial) output data
output_labels = output_sheet.range('A2:A6').expand('down').value
output_values = output_sheet.range('B2:B6').value

### Defining the Model
We will now create a function that drives the Excel model.

In [ ]:
# create "wrapper" around excel model
def simple_model_xl(Var1, Var2, Var3, Var4, Var5, Var6, Var7, Var8, Var9, Var10):
    input_sheet.range('B2:B11').options(transpose=True).value = [Var1, Var2, Var3, Var4, Var5, Var6, Var7, Var8, Var9, Var10]
    return output_sheet.range('B2:B6').value

### Setting up Rhodium

In [ ]:
# define model in rhodium
model_xl = Model(simple_model_xl)

model_xl.parameters = [
    Parameter( input_labels[0] ),
    Parameter( input_labels[1] ),
    Parameter( input_labels[2] ),
    Parameter( input_labels[3] ),
    Parameter( input_labels[4] ),
    Parameter( input_labels[5] ),
    Parameter( input_labels[6] ),
    Parameter( input_labels[7] ),
    Parameter( input_labels[8] ),
    Parameter( input_labels[9] )
] 

model_xl.levers = [
    IntegerLever( input_labels[0], input_mins[0], input_maxs[0] ),    
    IntegerLever( input_labels[1], input_mins[1], input_maxs[1] ),   
    IntegerLever( input_labels[2], input_mins[2], input_maxs[2] ),  
    IntegerLever( input_labels[3], input_mins[3], input_maxs[3] ), 
    IntegerLever( input_labels[4], input_mins[4], input_maxs[4] ),
    IntegerLever( input_labels[5], input_mins[5], input_maxs[5] ),
    IntegerLever( input_labels[6], input_mins[6], input_maxs[6] ),
    IntegerLever( input_labels[7], input_mins[7], input_maxs[7] ),
    IntegerLever( input_labels[8], input_mins[8], input_maxs[8] ),
    IntegerLever( input_labels[9], input_mins[9], input_maxs[9] )
]

model_xl.responses = [
    Response( output_labels[0], Response.MAXIMIZE ),
    Response( output_labels[1], Response.MINIMIZE ),
    Response( output_labels[2], Response.MAXIMIZE ),
    Response( output_labels[3], Response.MINIMIZE ),
    Response( output_labels[4], Response.MAXIMIZE )
]

# # alternate way to define model parameters
# # note: levers and responses result in type error (more work needed)
# for i, input in enumerate(input_labels):
#     foo = IntegerLever( input_labels[i], input_mins[i], input_maxs[i] )
#     model.parameters += Parameter( input_labels[i] )
#     model.levers += RealLever( input_labels[i], input_mins[i], input_maxs[i] )

# for i, output in enumerate(output_labels):
#     model.responses += Response( output_labels[i], Response.MAXIMIZE )


We can now perform the multi-objecive optimization using the NSGA-II genertic algorithm. Let's also time it for future reference.

In [ ]:
start_time = time.time()
output_xl = optimize(model, 'NSGAII', 1000)
runtime_xl = time.time() - start_time
print 'Runtime {:0.4f}s'.format(runtime_xl)

In [ ]:
df_output = output.as_dataframe()
df_output

In [ ]:
# save results to excel
database_sheet.range('A1').value = df_output

## Visualization

In [ ]:
# set up our graph styles
sns.set()
sns.set_style('darkgrid')

In [ ]:
parallel_coordinates(model, output, colormap='rainbow', target='top')

## Case #2: Using a Python Model

To best serve this example, we'll have to reverse engineer the simple model defined previously. We'll create a standalone -- admittedly contrived -- Python function that takes the expected inputs, performs the calculations, and returns the expected outputs.

In [ ]:
def simple_model_py(Var1, Var2, Var3, Var4, Var5, Var6, Var7, Var8, Var9, Var10):
    
    # handle some internal calculations
    Expression1 = Var1 * 0.336218764
    Expression2 = Var2 * 0.75222251
    Expression3 = Var3 * 0.358511648
    Expression4 = Var4 * 0.916881328
    Expression5 = Var5 * 0.968932684
    Expression6 = Var6 * 0.100469201
    Expression7 = Var7 * 0.008698836
    Expression8 = Var8 * 0.980040676
    Expression9 = Var9 * 0.36560712
    Expression10 = Var10 * 0.716986761    
    
    # evaluate outputs
    Output1 = Expression1 + Expression2 + Expression3 
    Output2 = Expression4 + Expression5 + Expression6
    Output3 = Expression7 + Expression8
    Output4 = Expression9 
    Output5 = Expression10
    
    return [ Output1, Output2, Output3, Output4, Output5 ]

### Sanity Check
Having defined the same model in two different ways, let's make sure they both behave the same way. We'll create a set of input parameters and evaluate them with each model. They should produce identical results.

In [ ]:
# define test inputs
Var1 = 5
Var2 = 2
Var3 = 5
Var4 = 2
Var5 = 5
Var6 = 2
Var7 = 5
Var8 = 2
Var9 = 5
Var10 = 2

# evaluate using different models
test1 = simple_model(Var1, Var2, Var3, Var4, Var5, Var6, Var7, Var8, Var9, Var10)
test2 = simple_model_py(Var1, Var2, Var3, Var4, Var5, Var6, Var7, Var8, Var9, Var10)

In [ ]:
# print formatted outputs and show their difference
print 'Test1: ' + '\t'.join( map(str, ['{:0.8f}'.format(val) for val in test1]) )
print 'Test2: ' + '\t'.join( map(str, ['{:0.8f}'.format(val) for val in test2]) )
print 'Diff:  ' + '\t'.join( map(str, ['{:0.8f}'.format(val) for val in np.subtract(test1, test2)]) )

The models are consistent with each other. As before, let's load the model into the Rhodium framework.

In [ ]:
model_py = Model(simple_model_py)

model_py.parameters = [
    Parameter( input_labels[0] ),
    Parameter( input_labels[1] ),
    Parameter( input_labels[2] ),
    Parameter( input_labels[3] ),
    Parameter( input_labels[4] ),
    Parameter( input_labels[5] ),
    Parameter( input_labels[6] ),
    Parameter( input_labels[7] ),
    Parameter( input_labels[8] ),
    Parameter( input_labels[9] )
] 

model_py.levers = [
    IntegerLever( input_labels[0], input_mins[0], input_maxs[0] ),    
    IntegerLever( input_labels[1], input_mins[1], input_maxs[1] ),   
    IntegerLever( input_labels[2], input_mins[2], input_maxs[2] ),  
    IntegerLever( input_labels[3], input_mins[3], input_maxs[3] ), 
    IntegerLever( input_labels[4], input_mins[4], input_maxs[4] ),
    IntegerLever( input_labels[5], input_mins[5], input_maxs[5] ),
    IntegerLever( input_labels[6], input_mins[6], input_maxs[6] ),
    IntegerLever( input_labels[7], input_mins[7], input_maxs[7] ),
    IntegerLever( input_labels[8], input_mins[8], input_maxs[8] ),
    IntegerLever( input_labels[9], input_mins[9], input_maxs[9] )
]

model_py.responses = [
    Response( output_labels[0], Response.MAXIMIZE ),
    Response( output_labels[1], Response.MINIMIZE ),
    Response( output_labels[2], Response.MAXIMIZE ),
    Response( output_labels[3], Response.MINIMIZE ),
    Response( output_labels[4], Response.MAXIMIZE )
]

In [ ]:
start_time = time.time()
output_py = optimize(model_py, 'NSGAII', 1000)
runtime_py = time.time() - start_time
print 'Runtime {:0.4f}s'.format(runtime_py)

**Note the significant gain in performance by running a Python model!**

In [ ]:
df_output_py = output_py.as_dataframe()
df_output_py

In [ ]:
parallel_coordinates(model_py, output_py, colormap='rainbow', target='top')

## Case #3: External Model
In some cases, models will be defined in another launguage such as C/C++, R, or Fortran.

### R
Let's try implementing our simple model in R. This will require the `pyper` package that can be installed by typing `pip install pyper` on the command line. Let's import Rhodium's R connector as follows

In [5]:
from rhodium.rbridge import RModel

In [6]:
model_r = RModel('simple_model.R', 'simple_model')

# the rest should look familiar by now...

model_r.parameters = [
    Parameter( input_labels[0] ),
    Parameter( input_labels[1] ),
    Parameter( input_labels[2] ),
    Parameter( input_labels[3] ),
    Parameter( input_labels[4] ),
    Parameter( input_labels[5] ),
    Parameter( input_labels[6] ),
    Parameter( input_labels[7] ),
    Parameter( input_labels[8] ),
    Parameter( input_labels[9] )
] 

model_r.levers = [
    IntegerLever( input_labels[0], input_mins[0], input_maxs[0] ),    
    IntegerLever( input_labels[1], input_mins[1], input_maxs[1] ),   
    IntegerLever( input_labels[2], input_mins[2], input_maxs[2] ),  
    IntegerLever( input_labels[3], input_mins[3], input_maxs[3] ), 
    IntegerLever( input_labels[4], input_mins[4], input_maxs[4] ),
    IntegerLever( input_labels[5], input_mins[5], input_maxs[5] ),
    IntegerLever( input_labels[6], input_mins[6], input_maxs[6] ),
    IntegerLever( input_labels[7], input_mins[7], input_maxs[7] ),
    IntegerLever( input_labels[8], input_mins[8], input_maxs[8] ),
    IntegerLever( input_labels[9], input_mins[9], input_maxs[9] )
]

model_r.responses = [
    Response( output_labels[0], Response.MAXIMIZE ),
    Response( output_labels[1], Response.MINIMIZE ),
    Response( output_labels[2], Response.MAXIMIZE ),
    Response( output_labels[3], Response.MINIMIZE ),
    Response( output_labels[4], Response.MAXIMIZE )
]

In [7]:
start_time = time.time()
output_r = optimize(model_r, 'NSGAII', 1000)
runtime_r = time.time() - start_time
print 'Runtime {:0.4f}s'.format(runtime_r)

LEVER:  <rhodium.model.IntegerLever object at 0x1131bc950>
LEVER:  <rhodium.model.IntegerLever object at 0x1131bcc10>
LEVER:  <rhodium.model.IntegerLever object at 0x1131bcc50>
LEVER:  <rhodium.model.IntegerLever object at 0x1131bcc90>
LEVER:  <rhodium.model.IntegerLever object at 0x1131bccd0>
LEVER:  <rhodium.model.IntegerLever object at 0x1131bcd10>
LEVER:  <rhodium.model.IntegerLever object at 0x1131bcd50>
LEVER:  <rhodium.model.IntegerLever object at 0x1131bcd90>
LEVER:  <rhodium.model.IntegerLever object at 0x1131bcdd0>
LEVER:  <rhodium.model.IntegerLever object at 0x1131bce10>
Runtime 12.2220s


In [ ]:
parallel_coordinates(model_r, output_r, colormap='rainbow', target='top')

### C
We've come this far, might as well try C. Our approach is somewhat different because it uses Rhodium's capability to run (Native Functions)[https://github.com/Project-Platypus/Rhodium/wiki/Models-in-Other-Languages#native-functions].  This requires the shared libraries generated during compile time. These libraries enable Rhodium to run specific functions in C, C++, or Fortran as if it were in natively in that environment -- hence the term.

In UNIX/Linux operating systems, these are files with `*.so` extensions. In Windows systems they're called dynamic link libraries and carry a `*.dll` extension.

For example, to create a shared library in a UNIX environment, one might do this:

```
gcc simple_model.c -shared -o simple_model.so
```

In [22]:
from rhodium.ffi import NativeModel

model_c = NativeModel('simple_model.so', 'simple_model')

model_c.parameters = [
    Parameter( input_labels[0], type='double' ),
    Parameter( input_labels[1], type='double' ),
    Parameter( input_labels[2], type='double' ),
    Parameter( input_labels[3], type='double' ),
    Parameter( input_labels[4], type='double' ),
    Parameter( input_labels[5], type='double' ),
    Parameter( input_labels[6], type='double' ),
    Parameter( input_labels[7], type='double' ),
    Parameter( input_labels[8], type='double' ),
    Parameter( input_labels[9], type='double' )
] 

model_c.levers = [
    IntegerLever( input_labels[0], input_mins[0], input_maxs[0] ),    
    IntegerLever( input_labels[1], input_mins[1], input_maxs[1] ),   
    IntegerLever( input_labels[2], input_mins[2], input_maxs[2] ),  
    IntegerLever( input_labels[3], input_mins[3], input_maxs[3] ), 
    IntegerLever( input_labels[4], input_mins[4], input_maxs[4] ),
    IntegerLever( input_labels[5], input_mins[5], input_maxs[5] ),
    IntegerLever( input_labels[6], input_mins[6], input_maxs[6] ),
    IntegerLever( input_labels[7], input_mins[7], input_maxs[7] ),
    IntegerLever( input_labels[8], input_mins[8], input_maxs[8] ),
    IntegerLever( input_labels[9], input_mins[9], input_maxs[9] )
]


# model_c.responses = [
#     Response( output_labels[0], type='double' ),
#     Response( output_labels[1], type='double' ),
#     Response( output_labels[2], type='double' ),
#     Response( output_labels[3], type='double' ),
#     Response( output_labels[4], type='double' )
# ]

model_c.responses = [
    Response('output', type='double*5' )
]

In [23]:
# start_time = time.time()
output_c = optimize(model_c, 'NSGAII', 1000)
# runtime_c = time.time() - start_time
# print 'Runtime {:0.4f}s'.format(runtime_c)

LEVER:  <rhodium.model.IntegerLever object at 0x1132b7bd0>
LEVER:  <rhodium.model.IntegerLever object at 0x113234050>
LEVER:  <rhodium.model.IntegerLever object at 0x113234850>
LEVER:  <rhodium.model.IntegerLever object at 0x1132340d0>
LEVER:  <rhodium.model.IntegerLever object at 0x113234150>
LEVER:  <rhodium.model.IntegerLever object at 0x113234290>
LEVER:  <rhodium.model.IntegerLever object at 0x113234650>
LEVER:  <rhodium.model.IntegerLever object at 0x1132346d0>
LEVER:  <rhodium.model.IntegerLever object at 0x113234f90>
LEVER:  <rhodium.model.IntegerLever object at 0x1132345d0>


In [24]:
output_c

[{u'Var1': 7,
  u'Var10': 1,
  u'Var2': 8,
  u'Var3': 7,
  u'Var4': 3,
  u'Var5': 4,
  u'Var6': 10,
  u'Var7': 6,
  u'Var8': 2,
  u'Var9': 6,
  'output': <rhodium.ffi.c_double_Array_5 at 0x1132ba170>}]

## Case #4: Other External Models
Occasionally a model will be black box, i.e. we (or Rhodium, for that matter) are unaware of the inner workings of the implementation but expect on the model to take input data and produce reliable output data. Examples of this are proprietary software, legacy binaries of which we no longer have access to the binary libraries, or binary software without an application programming interface, or API. The latter represents the worst-case scenario where a model was designed in a specific manner, e.g. one that requires user interaction via graphical user interface (GUI) and has no other designed implementation to provide inputs and outputs.

In some of these cases, the model may lend itself to reading and writing data to and from files, e.g. Excel/CSV. In such a case, 